In [1]:
import sys
import os
import re
import gensim
import numpy as np
import pandas as pd

import spacy # I might not need all the libraries out here -- just the ones initalized within the actual script 
nlp = spacy.load("en_core_web_sm")

os.chdir('/users/sbuongiorno/democracy-lab/util/pyfunctions/')

sys.path.append('../')
sys.path.append('../democracy-lab/util/pyfunctions/')
sys.path.append('../democracy-lab/word-embeddings/')

from pyfunctions.interval_subsetter import interval_subset
from pyfunctions.parallelize_operation import parallelize_operation
from pyfunctions.str_functions import lemmatize_df_text, str_split_df_sentences
from pyfunctions.gensim_w2v_functions import export_gensim_w2v_models, w2v_embeddings

os.chdir('/users/sbuongiorno/democracy-lab/word-embeddings')
sys.path.append('../hansard_subsets/')

In [5]:
hansard = pd.read_csv('~/hansard_justnine_w_year.csv')

hansard = hansard[['text','year']].copy()

hansard = hansard.rename(columns = {'text': 'speech'})

In [4]:
interval_subset(hansard, 'year', 1800, 1910, 5, fname='hansard')

In [5]:
%%time
export_gensim_w2v_models('hansard_subsets/', n_cores=12)

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 11.9 µs


KeyboardInterrupt: 

In [7]:
keyword = 'tenant'

In [8]:
keyword_context = w2v_embeddings.keyword_context('/users/sbuongiorno/democracy-lab/word-embeddings/hansard_subsets/', keyword)

In [62]:
#with open('propertywords_cleaned_for_word2vec.csv', newline='') as f:
#    reader = csv.reader(f)
#    keywords_list = list(reader)[1:] # skip the header

In [25]:
keyword_list = ['evict', 'tenant', 'landlord']

In [13]:
def define_periods(start, end, interval):
    periods = range(start, end, interval)
    
    period_names = []
    for period in periods:
        period_names.append(str(period) + '.0')
    
    return period_names

def collect_text_values(keyword_context, period_names):
    period_words = []
    
    for i in range(0, len(period_names)):
        try:
            words = [value[0] for value in keyword_context[i]]
            period_words.append(words)
        except:
            continue
    return period_words

        
def make_1D_list(period_words):
    flat_list = []
    
    for list in period_words:
        for word in list:
            flat_list.append(word)
    return flat_list

In [ ]:
period_names = define_periods(1800, 1905, 5)
period_words = collect_text_values(keyword_context, period_names)
flat_list = make_1D_list(period_words)

In [42]:
from matplotlib import cm
from numpy import linspace

colors = [ cm.gnuplot(x) for x in linspace(0, 1, len(flat_list)) ]

from adjustText import adjust_text
from matplotlib import pyplot as plt
%matplotlib inline
#from matplotlib.colors import ListedColormap, LinearSegmentedColormap

plt.figure(figsize=(30,30), dpi = 400)

texts = []

# plt.annotate only plots one label per iteration, so we have to use a for loop 
for i in range(0,len(period_names)):    # cycle through the period names                     
    for j in range(10):     # cycle through the first ten words (you can change this variable)
        if keyword_context[i]:
            xx = period_names[i]        # on the x axis, plot the period name
            yy = [item[1] for item in keyword_context[i]][j]         # on the y axis, plot the distance -- how closely the word is related to the keyword
            txt = [item[0] for item in keyword_context[i]][j]        # grab the name of each collocated word
            colorindex = flat_list.index(txt)                     # this command keeps all dots for the same word the same color
        
        
            plt.scatter(                                           # plot dots
                xx, #x axis
                yy, # y axis
                linewidth=1, 
                color = colors[colorindex],
                s = 300, # dot size
                alpha=0.5)  # dot transparency

            # make a label for each word
            texts.append(plt.text(xx, yy, txt))

# Code to help with overlapping labels -- may take a minute to run
adjust_text(texts, force_points=0.0001, force_text=0.0035, 
                    expand_points=(1, 1), expand_text=(1, 1),
                    arrowprops=dict(arrowstyle="-", color='black', lw=0.5))


plt.xticks(rotation=90)

# Add titles
plt.title("What words were most associated with ''" + keyword + "' in the Hansard debates?", fontsize=20, fontweight=0, color='Red')
plt.xlabel("period")
plt.ylabel("similarity to " + keyword)

plt.savefig(keyword + '.pdf')
plt.show()

#plt.savefig(keyword + '-over-time-' + str(startdate) + '-' + str(enddate) + '.pdf')